<a href="https://colab.research.google.com/github/Anvarka/RecSys/blob/master/HW_1_RECSYS_HSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! unzip /content/drive/MyDrive/ml-1m.zip
# ! pip install implicit

Archive:  /content/drive/MyDrive/ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [6]:
# !pip install lightfm
# import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

ModuleNotFoundError: ignored

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [7]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')
ratings

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [ ]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')
movie_info

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


Explicit данные

In [ ]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [8]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [9]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [ ]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [ ]:
model.fit(user_item_t_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [ ]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [ ]:
get_similars(1, model)

/usr/local/lib/python3.7/dist-packages/implicit/gpu/matrix_factorization_base.py:108: RuntimeWarning: invalid value encountered in true_divide
  scores /= self._item_norms_host[itemid]


['Series([], )',
 'Series([], )',
 'Series([], )',
 'Series([], )',
 'Series([], )',
 'Series([], )',
 'Series([], )',
 'Series([], )',
 'Series([], )',
 'Series([], )']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [ ]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [ ]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [ ]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [ ]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '2502    Matrix, The (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1892    Rain Man (1988)',
 '3402    Close Encounters of the Third Kind (1977)',
 '1179    Princess Bride, The (1987)',
 '847    Godfather, The (1972)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [102]:
import numpy as np
from tqdm import tqdm

In [84]:
ratings["rating"].mean()

3.581564453029317

In [89]:
movies = np.array(ratings["movie_id"].unique())
users = np.array(ratings["user_id"].unique())
dict_users = {}

for i in range(len(users)):
  dict_users[users[i]] = i

dict_movies = {} 
for i in range(len(movies)):
  dict_movies[movies[i]] = i

mov_users = np.full(shape=[len(users), len(movies)], fill_value=0)

movies = np.array(ratings["movie_id"])
users = np.array(ratings["user_id"])
rank = np.array(ratings["rating"])

for ind, row in ratings.iterrows():
    user_id = row["user_id"]
    movie_id = row["movie_id"]
    rating = row["rating"]
    i_ = dict_users[user_id]
    j_ = dict_movies[movie_id]

    mov_users[i_][j_] = rating

In [101]:
class GradientLR:
    def __init__(self, alpha:float, epochs=60, batch_size=100, hidden_dim=64):
        self.alpha = alpha
        self.epochs = epochs
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
    
    def fit(self, ratings):
        movies = np.array(ratings["movie_id"].unique())
        users = np.array(ratings["user_id"].unique())
        
        self.u = np.random.normal(loc=0, scale=1, size=(len(users), self.hidden_dim))
        self.v = np.random.normal(loc=0, scale=1, size=(len(movies), self.hidden_dim))

        for i in tqdm(range(self.epochs)):
            score = (self.u @ self.v.T - mov_users + 3.5 + 1 + 2)

            delta_L_u = 2 * score @ self.v + 2 * self.u * 0.001
            delta_L_v = 2 * score.T @ self.u + 2 * self.v * 0.001

            self.u -= self.alpha * delta_L_u
            self.v -= self.alpha * delta_L_v
            print(np.mean(score ** 2))

In [92]:
import numpy as np
movies = ratings["movie_id"]
rating = ratings["rating"]

regr = GradientLR(alpha=0.0001, epochs=100, batch_size=100)
regr.fit(ratings)

  1%|          | 1/100 [00:00<01:00,  1.63it/s]

64.30321108797044


  2%|▏         | 2/100 [00:01<00:58,  1.67it/s]

42.612182980148624


  3%|▎         | 3/100 [00:01<00:59,  1.63it/s]

...

0.5708043812352348


 99%|█████████▉| 99/100 [00:58<00:00,  1.67it/s]

0.5686199359479002


100%|██████████| 100/100 [00:59<00:00,  1.68it/s]

0.5664310631004319


array([nan, nan, nan, ..., nan, nan, nan])

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [98]:
movies = np.array(implicit_ratings["movie_id"].unique())
users = np.array(implicit_ratings["user_id"].unique())
dict_users = {}

for i in range(len(users)):
  dict_users[users[i]] = i

dict_movies = {} 
for i in range(len(movies)):
  dict_movies[movies[i]] = i

mov_users_implicit = np.full(shape=[len(users), len(movies)], fill_value=0)

movies = np.array(implicit_ratings["movie_id"])
users = np.array(implicit_ratings["user_id"])
rank = np.array(implicit_ratings["rating"])

for ind, row in implicit_ratings.iterrows():
    user_id = row["user_id"]
    movie_id = row["movie_id"]
    rating = row["rating"]
    i_ = dict_users[user_id]
    j_ = dict_movies[movie_id]

    mov_users_implicit[i_][j_] = rating

In [99]:
class GradientLR:
    def __init__(self, alpha:float, epochs=60, batch_size=100, hidden_dim=64):
        self.alpha = alpha
        self.epochs = epochs
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
    
    def fit(self, ratings):
        movies = np.array(ratings["movie_id"].unique())
        users = np.array(ratings["user_id"].unique())
        
        self.u = np.random.normal(loc=0, scale=1, size=(len(users), self.hidden_dim))
        self.v = np.random.normal(loc=0, scale=1, size=(len(movies), self.hidden_dim))

        for i in tqdm(range(self.epochs)):
            score = (self.u @ self.v.T - mov_users_implicit)

            if i % 2 == 0:
                delta_L_u = 2 * score @ self.v + 2 *self.u * 0.001
                delta_L_v = 0
            else:
                delta_L_u = 0
                delta_L_v = 2 * score.T @ self.u + 2 *self.v * 0.001

            self.u -= self.alpha * delta_L_u
            self.v -= self.alpha * delta_L_v
            print(np.mean(score ** 2))

In [100]:
import numpy as np

regr = GradientLR(alpha=0.0001, epochs=100, batch_size=100)
regr.fit(implicit_ratings)

  1%|          | 1/100 [00:00<00:37,  2.64it/s]

64.35386147343169


  2%|▏         | 2/100 [00:00<00:38,  2.56it/s]

6.2193106989042075


  3%|▎         | 3/100 [00:01<00:37,  2.59it/s]

...

0.4037416757845133


 99%|█████████▉| 99/100 [00:38<00:00,  2.58it/s]

0.40322897834249677


100%|██████████| 100/100 [00:38<00:00,  2.57it/s]

0.40314989625643577


### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [12]:
from collections import defaultdict
# movies = np.array(implicit_ratings["movie_id"])
# users = np.array(implicit_ratings["user_id"])


movies = np.array(ratings["movie_id"].unique())
users = np.array(ratings["user_id"].unique())

mov_users_data = np.full(shape=[len(users), len(movies)], fill_value=0)

dict_users = {}
for i in range(len(users)):
  dict_users[users[i]] = i

dict_movies = {} 
for i in range(len(movies)):
  dict_movies[movies[i]] = i


positives = defaultdict(list)
negatives = defaultdict(list)

for user_id, df in ratings.groupby("user_id"):
    positives[user_id] = np.array(list([df[df['rating'] >= 4]['movie_id'].values])).flatten()
    negatives[user_id] = np.array(list([df[df['rating'] < 4]['movie_id'].values])).flatten()

In [165]:
mov_users_data.shape

(6040, 3706)

In [10]:
import numpy as np
from tqdm import tqdm


In [135]:
class GradientLR:
    def __init__(self, alpha:float, epochs=60, lambda_=0.01, hidden_dim=64):
        self.alpha = alpha
        self.epochs = epochs
        self.hidden_dim = hidden_dim
        self.lambda_ = lambda_

    def get_negative_item(self, user, movies, count):
        if len(negatives[user]) == 0:
            for movie in movies:
                if not movie in positives[user]:
                    negatives[user] = np.array([movie])
                    break

        return np.random.choice(negatives[user], count)

    def get_positive_item(self, user, movies, count):
        if len(positives[user]) == 0:
            for movie in movies:
                if not movie in negatives[user]:
                    positives[user] = np.array([movie])
                    break

        return np.random.choice(positives[user], count)

    def get_correct_item_ids(self, user, movies, dict_movies, count):

        positive_ids = self.get_positive_item(user, movies, count=count)
        negative_ids = self.get_negative_item(user, movies, count=count)

        positive_ids = np.array(list(dict_movies[k] for k in positive_ids))
        negative_ids = np.array(list(dict_movies[k] for k in negative_ids))
        return positive_ids, negative_ids


    def fit(self, ratings):
        movies = np.array(ratings["movie_id"].unique())

        dict_movies = {}
        for i in  range(len(movies)):
            dict_movies[movies[i]] = i

        users = np.array(ratings["user_id"].unique())

        positives_emb = np.random.normal(size=(len(users), self.hidden_dim))
        self.u = np.random.normal(loc=0, scale=1, size=(len(users), self.hidden_dim))
        self.v = np.random.normal(loc=0, scale=1, size=(len(movies), self.hidden_dim))

        for i in tqdm(range(self.epochs)): 

            positive_user_items = {}
            negative_user_items = {} 

            # sum exp(-x)/(1+exp(-x)) = -1/(exp(sum(x))+1)
            # sum(x_uij) = sum(x_ui - x_uj)
            # x_ui = U @ V_pos.T
            # x_uj = U @ V_neg.T
            sum_diff = 0
            for j in range(len(users)):
                positive_ids, negative_ids = self.get_correct_item_ids(users[j], movies, dict_movies, count=32)

                positive_user_items[j] = positive_ids
                negative_user_items[j] = negative_ids

                sum_diff += self.u[j] @ (self.v[positive_ids] - self.v[negative_ids]).T
            sum_diff = np.array(sum_diff, dtype=np.float128)
            numerator = 1.0 / (1.0 + np.exp(sum_diff))

            for j in range(len(users)):
                score = self.u @ self.v.T

                score_positive = self.v[positive_user_items[j]]
                score_negative = self.v[negative_user_items[j]]

                temp = self.u[j]
                # if theta = U    
                differential = score_positive - score_negative
                
                self.u[j] += self.alpha * (numerator.T @ differential - self.lambda_ * temp)

                # if theta = score_pos
                differential = temp
                self.v[positive_ids] = self.v[positive_ids] + self.alpha * (np.outer(numerator, differential.T) - self.lambda_ * score_positive)

                # if theta = score_pos
                differential = -temp
                self.v[negative_ids] = self.v[negative_ids] + self.alpha * (-np.outer(numerator, differential.T) - self.lambda_ * score_negative)
                    
        print(np.mean(score**2))

In [ ]:
import numpy as np

regr = GradientLR(alpha=0.01, epochs=100)
regr.fit(ratings)

  0%|          | 0/100 [00:00<?, ?it/s]

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных